# DaCe IR – Library Nodes and Operator Abstractions

This tutorial demonstrates how DaCe's Intermediate Representation (IR) handles operator abstractions through LibraryNodes. We'll explore:

1. **LibraryNodes**: High-level operator abstractions in the IR
2. **Semantic Contracts**: How `@dace.library.node` defines compile-time "facts"
3. **Implementation Registry**: How concrete implementations are attached to abstract operators
4. **Expansion System**: How abstract operators become concrete code


In [2]:
import dace
import numpy as np
from dace import library, SDFG, SDFGState, symbolic, dtypes
from dace.sdfg import nodes
from dace.transformation.transformation import ExpandTransformation

# Print DaCe version for reference
print(f"DaCe version: {dace.__version__}")
print(f"LibraryNode base class: {dace.sdfg.nodes.LibraryNode}")

DaCe version: 1.0.0
LibraryNode base class: <class 'dace.sdfg.nodes.LibraryNode'>



## What are LibraryNodes?

LibraryNodes are DaCe's way of representing high-level operations (like GEMM, FFT, reductions) as single nodes in the SDFG. They serve as:
- **Abstractions**: Hide implementation complexity behind semantic interfaces
- **Contracts**: Define what the operation does, not how it's implemented
- **Extension Points**: Allow multiple implementations for different targets (CPU, GPU, FPGA)

## 1. Understanding Library Nodes in DaCe IR

### What are LibraryNodes?

In DaCe, a **Library Node** is an abstract representation of an operation in the SDFG. Instead of immediately expanding into low-level code, the Library Node represents a high-level operation with well-defined inputs and outputs.

Key benefits:
- **Deferred Implementation**: The operation can be realized in different ways (native code, library calls)
- **Semantic Transformations**: High-level optimizations based on operation semantics
- **Performance Portability**: Same algorithm can target different hardware by switching implementations

Library Nodes serve as the semantic contract for an operation, defining:
1. Input/output connectors
2. Compile-time properties and parameters 
3. Available implementations

### How to Define a Library Node

Library Nodes are created by decorating a class with `@dace.library.node` (establishes the semantic contracts):

In [3]:
# This is a simplified example of how a library node is defined
@library.node
class ExampleNode(nodes.LibraryNode):
    # Available implementations (expansions) for this node
    implementations = {}
    # Default implementation name
    default_implementation = None
    
    # Define properties that capture compile-time "facts"
    alpha = dace.properties.Property(dtype=float, default=1.0)
    beta = dace.properties.Property(dtype=float, default=0.0)
    
    def __init__(self, name, alpha=1.0, beta=0.0):
        # Define input/output interface
        super().__init__(name, 
                         inputs={"A", "B"},  # Input connectors
                         outputs={"C"})      # Output connectors
        # Set properties
        self.alpha = alpha
        self.beta = beta
    
    def validate(self, sdfg, state):
        # Perform semantic validation
        # (e.g., ensure shapes are compatible)
        pass

print("Example library node class defined")

Example library node class defined


## 3. Exploring Matrix Multiplication Library Node Example

Let's examine a real Library Node: DaCe's built-in MatMul (matrix multiplication) node. 
First, we'll look at the node's structure and properties:

In [4]:
# Import the MatMul library node from DaCe's BLAS library
from dace.libraries.blas.nodes.matmul import MatMul

# Create an instance to inspect
mm = MatMul("example_matmul")

print("=== MatMul LibraryNode Analysis ===")
print(f"Class: {MatMul}")
print(f"Base classes: {MatMul.__bases__}")
print(f"Is a LibraryNode: {issubclass(MatMul, dace.sdfg.nodes.LibraryNode)}")

# Examine instance properties
print(f"\nInstance name: {mm.name}")
print(f"Instance label: {mm.label}")
print(f"Alpha: {mm.alpha}")
print(f"Beta: {mm.beta}")
print(f"Implementation: {mm.implementation}")
print(f"Schedule: {mm.schedule}")

# Check input/output interface
print(f"\nInput connectors: {mm.in_connectors}")
print(f"Output connectors: {mm.out_connectors}")

# Check available implementations
print(f"\n=== Available Implementations ===")
for impl_name, impl_class in MatMul.implementations.items():
    print(f"  {impl_name}: {impl_class}")

print(f"\nDefault implementation: {MatMul.default_implementation}")

# Check library registration
print(f"\n=== Library Registration ===")
print(f"Library name: {getattr(MatMul, '_dace_library_name', 'Not registered')}")
print(f"Is library node: {getattr(MatMul, '_dace_library_node', False)}")

=== MatMul LibraryNode Analysis ===
Class: <class 'dace.libraries.blas.nodes.matmul.MatMul'>
Base classes: (<class 'dace.sdfg.nodes.LibraryNode'>,)
Is a LibraryNode: True

Instance name: example_matmul
Instance label: example_matmul
Alpha: 1
Beta: 0
Implementation: None
Schedule: ScheduleType.Default

Input connectors: {'_b': void, '_a': void}
Output connectors: {'_c': void}

=== Available Implementations ===
  specialize: <class 'dace.libraries.blas.nodes.matmul.SpecializeMatMul'>

Default implementation: specialize

=== Library Registration ===
Library name: blas
Is library node: True


Now let's see MatMul in action by creating a simple matrix multiplication program using DaCe:

In [5]:
# Symbolic dimension (for generality)
N = dace.symbol('N')

# Define a DaCe program for matrix multiplication
@dace.program
def matmul_example(A: dace.float32[N, N], B: dace.float32[N, N]):
    return A @ B  # Using the @ operator triggers a MatMul library node

# Instantiate example data
N_val = 4
A = np.random.rand(N_val, N_val).astype(np.float32)
B = np.random.rand(N_val, N_val).astype(np.float32)

# Get the SDFG representation
sdfg = matmul_example.to_sdfg()
sdfg.save('matmul_example.sdfg')  # Save to file (optional, for visualization)
print("SDFG states:", sdfg.states)
print("SDFG states:", len(sdfg.states()))
state = sdfg.states()[0]

print("\nNodes in SDFG state:")
# Find the library node in the state
for node in state.nodes():
    print(type(node), ":", node.label, "| Implementation:", getattr(node, "implementation", None))

SDFG states: <bound method SDFG.states of SDFG (matmul_example)>
SDFG states: 1

Nodes in SDFG state:
<class 'dace.sdfg.nodes.AccessNode'> : A | Implementation: None
<class 'dace.sdfg.nodes.AccessNode'> : B | Implementation: None
<class 'dace.sdfg.nodes.AccessNode'> : __return | Implementation: None
<class 'dace.libraries.blas.nodes.matmul.MatMul'> : _MatMult_ | Implementation: None


/Users/sophieblock/dev_packages/dace/dace/transformation/dataflow/map_fusion_vertical.py:410: SyntaxWarning: invalid escape sequence '\m'
  * Pure Output Set `\mathbb{P}`:


### Different Implementation Options for MatMul

Let's explore different expansion options for MatMul:

In [6]:
# 1. Use default "pure" expansion
sdfg_pure = matmul_example.to_sdfg()
sdfg_pure.specialize({'N': N_val})  # Replace symbol N with its concrete value

# Expand with the default implementation
sdfg_pure.expand_library_nodes()
print("Expanded with 'pure' implementation:")
print(dir(sdfg_pure))
print(sdfg_pure.states())
print(f"  States: {len(sdfg_pure.states())}")
print(f"  Nodes in state 0: {len(sdfg_pure.states()[0].nodes())}")
print(f"  Node types: {[type(n).__name__ for n in sdfg_pure.states()[0].nodes()][:5]}...")

# 2. Try MKL implementation if available
sdfg_mkl = matmul_example.to_sdfg()
sdfg_mkl.specialize({'N': N_val})  # Replace symbol N with its concrete value

# Find the MatMul node and set implementation to MKL
matmul_node = None
for node in sdfg_mkl.states()[0].nodes():
    if isinstance(node, dace.libraries.blas.nodes.MatMul):
        matmul_node = node
        break

if matmul_node:
    try:
        from dace.libraries.blas import MKL
        if hasattr(MKL, 'is_installed') and MKL.is_installed():
            print("\nMKL is available, setting implementation to 'MKL'")
            matmul_node.implementation = "MKL"
            sdfg_mkl.expand_library_nodes()
            print(f"Expanded with 'MKL' implementation:")
            print(f"  States: {len(sdfg_mkl.states())}")
            print(f"  Nodes in state 0: {len(sdfg_mkl.states()[0].nodes())}")
        else:
            print("\nMKL is not available in this environment")
    except ImportError:
        print("\nMKL is not available in this environment")
else:
    print("\nMatMul node not found in SDFG")

# Run the SDFG with our data
print("\nRunning the matrix multiplication:")
result = sdfg_pure(A=A, B=B)
print("A shape:", A.shape)
print("B shape:", B.shape)
print("Result shape:", result.shape)
print("First few values of result:", result.flatten()[:4], "...")

# Verify against numpy
np_result = A @ B
print("Matches numpy:", np.allclose(result, np_result))

Automatically expanded library node "_MatMult_" with implementation "specialize".
Automatically expanded library node "_MatMult_gemm" with implementation "pure".
Expanded with 'pure' implementation:
['__abstractmethods__', '__annotations__', '__arrays', '__call__', '__class__', '__class_getitem__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__pgrids', '__properties__', '__rdistrarrays', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__static_attributes__', '__str__', '__subarrays', '__subclasshook__', '__weakref__', '_abc_impl', '_arg_names', '_arrays', '_cached_start_block', '_callback_mapping', '_cfg_list', '_constants_prop', '_debuginfo', '_default_linei

/Users/sophieblock/dev_packages/dace/dace/sdfg/validation.py:273: UserWarning: Mismatch between constant and symbol type of "N", expected to find "Scalar (dtype=int64_t)" but found "int".
  warnings.warn(f'Mismatch between constant and symbol type of "{const_name}", '


-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/sophieblock/dev_packages/dace/tutorials/.dacecache/matmul_example/build

-- Generating done (0.0s)
-- Build files have been written to: /Users/sophieblock/dev_packages/dace/tutorials/.dacecache/matmul_example/build

[ 25%] Building CXX object CMakeFiles/matmul_example.dir/Users/sophieblock/dev_packages/dace/tutorials/.dacecache/matmul_example/src/cpu/matmul_example.cpp.o
[ 25%] Building CXX object CMakeFiles/matmul_example.dir/Users/sophieblock/dev_packages/dace/tutorials/.dacecache/matmul_example/src/cpu/matmul_example.cpp.o
[ 50%] Linking CXX shared library libmatmul_example.dylib
[ 50%] Linking CXX shared library libmatmul_example.dylib
[ 50%] Built target matmul_example
[ 50%] Built target matmul_example
[100%] Built target dacestub_matmul_example

[100%] Built target dacestub_matmul_example

A shape: (4, 4)
B shape: (4, 4)
Result shape: (4, 4)
First few values of result: [0.6323359  0.

## 4. Creating a Custom Matrix Inversion Library Node

Now let's create our own library node for matrix inversion. We'll define:
1. A `MatInv` LibraryNode class with input "A" and output "Inv" 
2. Two implementations: direct (Gaussian elimination) and iterative (Conjugate Gradient)

In [29]:
# Define the MatInv library node
@library.node
class MatInv(dace.sdfg.nodes.LibraryNode):
    """
    A LibraryNode that computes the inverse of a matrix: Inv = A^(-1)
    """
    # Register available implementations (will populate after classes are defined)
    implementations = {}
    default_implementation = None  # we'll set a default later

    def __init__(self, name=None):
        # Define input/output interface
        super().__init__(name or "MatInv", 
                         inputs={"A"},      # Input matrix to invert
                         outputs={"Inv"})   # Output inverse matrix

    def validate(self, sdfg, state):
        """
        Validate usage - for example, we could check that A is square
        """
        # Find input array descriptor
        for e in state.in_edges(self):
            if e.dst_conn == 'A':
                arr = sdfg.arrays[e.data.data]
                if len(arr.shape) != 2 or arr.shape[0] != arr.shape[1]:
                    raise ValueError(f"MatInv requires a square matrix, got shape {arr.shape}")

print("MatInv library node defined")
print(f"Is MatInv a LibraryNode? {issubclass(MatInv, dace.sdfg.nodes.LibraryNode)}")
print(f"Input connectors: {MatInv().in_connectors}")
print(f"Output connectors: {MatInv().out_connectors}")

MatInv library node defined
Is MatInv a LibraryNode? True
Input connectors: {'A': void}
Output connectors: {'Inv': void}


## 5. Implementing Gaussian Elimination Expansion

Let's implement our first expansion using Gaussian elimination for direct matrix inversion.
First, we define a DaCe program for Gaussian elimination:

In [30]:
# Define symbolic size for our matrices
N = dace.symbol('N')

@dace.program
def invert_gaussian(A: dace.float64[N, N]):
    """
    Compute matrix inverse using Gaussian elimination with Gauss-Jordan method
    """
    # Make local copies to avoid modifying input
    A_work = np.copy(A)         # Working copy of A (will be transformed to identity)
    Inv = np.eye(N, dtype=A.dtype)  # Start with Inv as identity matrix
    
    # Gauss-Jordan elimination
    for i in range(N):
        # Divide i-th row by the pivot A_work[i, i]
        pivot = A_work[i, i]
        for j in range(N):
            A_work[i, j] = A_work[i, j] / pivot
            Inv[i, j] = Inv[i, j] / pivot
            
        # Eliminate column i in all other rows
        for k in range(N):
            if k != i:
                factor = A_work[k, i]
                for j in range(N):
                    A_work[k, j] = A_work[k, j] - factor * A_work[i, j]
                    Inv[k, j] = Inv[k, j] - factor * Inv[i, j]
    
    return Inv

# Test the Gaussian elimination function
A_test = np.array([[4, 7], [2, 6]], dtype=np.float64)
print("Test matrix A:")
print(A_test)

# Convert our DaCe program to SDFG and test it
gaussian_sdfg = invert_gaussian.to_sdfg()
# Set a concrete value for N
gaussian_sdfg.specialize({'N': 2})
gaussian_func = gaussian_sdfg.compile()
inv_result = gaussian_func(A=A_test)

print("\nInverse using our Gaussian elimination:")
print(inv_result)
print("\nInverse using numpy.linalg.inv:")
print(np.linalg.inv(A_test))
print("\nVerification: A * A^(-1) should be close to identity:")
print(A_test @ inv_result)

Test matrix A:
[[4. 7.]
 [2. 6.]]


/Users/sophieblock/dev_packages/dace/dace/sdfg/sdfg.py:2408: UserWarning: SDFG 'invert_gaussian' is already loaded by another object, recompiling under a different name 'invert_gaussian_0'.
  warnings.warn(f"SDFG '{self.name}' is already loaded by another object, recompiling under a different "


-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_gaussian/build

-- Build files have been written to: /Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_gaussian/build

[ 25%] Building CXX object CMakeFiles/invert_gaussian_0.dir/Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_gaussian/src/cpu/invert_gaussian_0.cpp.o
[ 25%] Building CXX object CMakeFiles/invert_gaussian_0.dir/Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_gaussian/src/cpu/invert_gaussian_0.cpp.o
[ 50%] Linking CXX shared library libinvert_gaussian_0.dylib
[ 50%] Linking CXX shared library libinvert_gaussian_0.dylib
[ 50%] Built target invert_gaussian_0
[ 50%] Built target invert_gaussian_0
[ 75%] Building CXX object CMakeFiles/dacestub_invert_gaussian_0.dir/tools/dacestub.cpp.o
[ 75%] Building CXX object CMakeFiles/dacestub_invert_gaussian_0.di

Now, let's create an expansion class for our Gaussian elimination implementation:

In [31]:
# Define expansion class for Gaussian elimination
@library.register_expansion(MatInv, 'gaussian')
class ExpandMatInvGaussian(ExpandTransformation):
    """
    A direct matrix inversion implementation using Gaussian elimination
    """
    environments = []  # no special environment needed (pure Python implementation)

    @staticmethod
    def expansion(node: MatInv, parent_state: dace.SDFGState, parent_sdfg: dace.SDFG, **kwargs):
        """
        Expansion method: replaces the MatInv node with a nested SDFG
        that implements Gaussian elimination
        """
        print(f"=== Expanding MatInv node with Gaussian elimination ===")
        print(f"Node name: {node.name}")
        
        # Determine N value from input matrix
        N_val = None
        for e in parent_state.in_edges(node):
            if e.dst_conn == 'A':
                arr = parent_sdfg.arrays[e.data.data]
                if isinstance(arr.shape[0], int):
                    N_val = arr.shape[0]
        
        # Create an SDFG for the inversion using our DaCe program
        inv_sdfg = invert_gaussian.to_sdfg()
        inv_sdfg.name = f"expand_matinv_gaussian_{node.label}"
        
        # Specialize with concrete value for N if available
        if N_val is not None:
            inv_sdfg.specialize({'N': N_val})
        
        # Rename return array to match our connector name
        if "__return" in inv_sdfg.arrays:
            inv_sdfg.arrays["Inv"] = inv_sdfg.arrays.pop("__return")
            # Update references in the SDFG
            for st in inv_sdfg.states():
                for n in st.nodes():
                    if isinstance(n, dace.sdfg.nodes.AccessNode) and n.data == "__return":
                        n.data = "Inv"
        
        print(f"Created nested SDFG for Gaussian elimination")
        return inv_sdfg

# Register the implementation in our MatInv class
MatInv.default_implementation = 'gaussian'

print(f"Registered 'gaussian' implementation for MatInv")
print(f"Available implementations: {list(MatInv.implementations.keys())}")
print(f"Default implementation: {MatInv.default_implementation}")

Registered 'gaussian' implementation for MatInv
Available implementations: ['gaussian']
Default implementation: gaussian


## 6. Implementing Iterative (Conjugate Gradient) Expansion

Let's implement a second expansion using an iterative Conjugate Gradient approach.
This is an alternative method for matrix inversion that's especially efficient for
sparse matrices and can be more suitable for certain hardware.

In [ ]:
@dace.program
def invert_cg(A: dace.float64[N, N], maxiter: dace.int32):
    """
    Compute matrix inverse using Conjugate Gradient method.
    Solves A * x = e_i for each column of the identity matrix.
    """
    Inv = np.zeros((N, N), dtype=A.dtype)
    
    # Solve A * x = e_i for each column i
    for i in range(N):
        # Initialize for CG
        b = np.zeros((N,), dtype=A.dtype)
        b[i] = 1.0  # unit vector (column of identity)
        
        x = np.zeros((N,), dtype=A.dtype)  # initial guess
        r = np.copy(b)                     # initial residual = b (since x=0)
        p = np.copy(r)                     # initial direction
        rr = np.dot(r, r)                  # r^T r
        
        # CG iterations
        for k in range(maxiter):
            Ap = A @ p
            alpha = rr / np.dot(p, Ap)
            
            x = x + alpha * p
            r = r - alpha * Ap
            
            new_rr = np.dot(r, r)
            if new_rr < 1e-12:
                break  # converged
                
            p = r + (new_rr/rr) * p
            rr = new_rr
            
        # Store solution x as column i of Inv
        for j in range(N):
            Inv[j, i] = x[j]
            
    return Inv

# Test the CG inversion on our test matrix
# Make it symmetric positive-definite for better convergence
A_test_spd = A_test @ A_test.T + np.eye(2)  # ensures positive definiteness
print("Symmetric positive-definite test matrix:")
print(A_test_spd)

# Test with explicit maxiter
cg_sdfg = invert_cg.to_sdfg()
# Specialize with concrete values
cg_sdfg.specialize({'N': 2})
# Remove the maxiter symbol and add it as a constant
if 'maxiter' in cg_sdfg.symbols:
    cg_sdfg.remove_symbol('maxiter')
cg_sdfg.add_constant('maxiter', 10)  # Set max iterations
cg_func = cg_sdfg.compile()
# Now we can call the function without passing maxiter
cg_result = cg_func(A=A_test_spd)

print("\nInverse using Conjugate Gradient (10 iterations):")
print(cg_result)
print("\nInverse using numpy.linalg.inv:")
print(np.linalg.inv(A_test_spd))
print("\nVerification: A * A^(-1) should be close to identity:")
print(A_test_spd @ cg_result)

Symmetric positive-definite test matrix:
[[66. 50.]
 [50. 41.]]


/Users/sophieblock/dev_packages/dace/dace/sdfg/sdfg.py:2408: UserWarning: SDFG 'invert_cg' is already loaded by another object, recompiling under a different name 'invert_cg_1'.
  warnings.warn(f"SDFG '{self.name}' is already loaded by another object, recompiling under a different "


Automatically expanded library node "_MatMult_" with implementation "specialize".
Automatically expanded library node "_Dot_" with implementation "pure".
Automatically expanded library node "_Dot_" with implementation "pure".
Automatically expanded library node "_MatMult_gemv" with implementation "pure".
Automatically expanded library node "_Dot_" with implementation "pure".
-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_cg/build

-- Build files have been written to: /Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_cg/build

[ 25%] Building CXX object CMakeFiles/invert_cg_1.dir/Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_cg/src/cpu/invert_cg_1.cpp.o
[ 25%] Building CXX object CMakeFiles/invert_cg_1.dir/Users/sophieblock/dev_packages/dace/tutorials/.dacecache/invert_cg/src/cpu/invert_cg_1.cpp.

Now, let's create the expansion class for our Conjugate Gradient implementation:

In [35]:
@library.register_expansion(MatInv, 'iterative')
class ExpandMatInvIterative(ExpandTransformation):
    """
    An iterative matrix inversion implementation using Conjugate Gradient
    """
    environments = []  # no special environment needed

    @staticmethod
    def expansion(node: MatInv, parent_state: dace.SDFGState, parent_sdfg: dace.SDFG, **kwargs):
        """
        Expansion method: replaces the MatInv node with a nested SDFG
        that implements Conjugate Gradient
        """
        print(f"=== Expanding MatInv node with Conjugate Gradient ===")
        print(f"Node name: {node.name}")
        
        # Determine a reasonable maxiter based on matrix size
        N_val = None
        for e in parent_state.in_edges(node):
            if e.dst_conn == 'A':
                arr = parent_sdfg.arrays[e.data.data]
                if isinstance(arr.shape[0], int):
                    N_val = arr.shape[0]
                    
        # Use matrix size as max iterations (optimal for CG on N×N matrix),
        # or a reasonable default if size is symbolic
        maxiter = N_val if N_val is not None else 1000
        
        # Create SDFG from the CG program
        inv_sdfg = invert_cg.to_sdfg()
        inv_sdfg.name = f"expand_matinv_iterative_{node.label}"
        
        # Specialize the SDFG with concrete value for N
        if N_val is not None:
            inv_sdfg.specialize({'N': N_val})
            
        # IMPORTANT: Remove maxiter from arguments by making it a constant
        if 'maxiter' in inv_sdfg.symbols:
            inv_sdfg.remove_symbol('maxiter')
        inv_sdfg.add_constant('maxiter', maxiter)
        
        # Rename return array to match our connector name
        if "__return" in inv_sdfg.arrays:
            inv_sdfg.arrays["Inv"] = inv_sdfg.arrays.pop("__return")
            # Update references in the SDFG
            for st in inv_sdfg.states():
                for n in st.nodes():
                    if isinstance(n, dace.sdfg.nodes.AccessNode) and n.data == "__return":
                        n.data = "Inv"
        
        print(f"Created nested SDFG for Conjugate Gradient with maxiter={maxiter}")
        return inv_sdfg

print(f"Registered 'iterative' implementation for MatInv")
print(f"Available implementations: {list(MatInv.implementations.keys())}")

# Show the implementation registry
print(f"\n=== Implementation Registry ===")
for name, impl_class in MatInv.implementations.items():
    print(f"  {name}: {impl_class}")
    print(f"    - Environments required: {impl_class.environments}")
    print(f"    - Associated node: {getattr(impl_class, '_dace_library_node', 'Not set')}")

Registered 'iterative' implementation for MatInv
Available implementations: ['gaussian', 'iterative']

=== Implementation Registry ===
  gaussian: <class '__main__.ExpandMatInvGaussian'>
    - Environments required: []
    - Associated node: Not set
  iterative: <class '__main__.ExpandMatInvIterative'>
    - Environments required: []
    - Associated node: Not set


## 7. Using the Matrix Inversion Library Node

Now that we have defined our custom MatInv library node with two implementations,
let's use it by creating an SDFG manually:

In [36]:
# Create a sample matrix to invert
N_val = 4
A_val = np.random.randn(N_val, N_val).astype(np.float64)
# Ensure it's invertible (add identity matrix to avoid singular matrices)
A_val += 0.5 * np.eye(N_val)

# Construct SDFG for matrix inversion
sdfg = dace.SDFG('invert_matrix')
state = sdfg.add_state()

# Add data descriptors
sdfg.add_array('A', shape=[N_val, N_val], dtype=dace.float64)
sdfg.add_array('Inv_out', shape=[N_val, N_val], dtype=dace.float64)

# Add access nodes for data
A_read = state.add_read('A')
Inv_write = state.add_write('Inv_out')

# Add our MatInv library node
inv_node = MatInv("my_matrix_inverse")
state.add_node(inv_node)

# Connect input and output
state.add_edge(A_read, None, inv_node, 'A', dace.Memlet('A'))  # A -> node.A
state.add_edge(inv_node, 'Inv', Inv_write, None, dace.Memlet('Inv_out'))  # node.Inv -> Inv_out

# By default, this MatInv will use the 'gaussian' expansion
print(f"MatInv node implementation: {inv_node.implementation or 'default'}")

# Compile and run the SDFG
print("Compiling SDFG with Gaussian elimination implementation...")
inv_func = sdfg.compile()
result = inv_func(A=A_val)

# Verify the result
print("\nOriginal matrix A (first 2x2 corner):")
print(A_val[:2, :2])
print("\nComputed inverse (first 2x2 corner):")
print(result['Inv_out'][:2, :2])
print("\nA * A^(-1) (should be close to identity):")
print((A_val @ result['Inv_out'])[:4, :4])

# Compare with NumPy's inverse
numpy_inv = np.linalg.inv(A_val)
print("\nNumPy inverse (first 2x2 corner):")
print(numpy_inv[:2, :2])
print("\nMatch with NumPy: ", np.allclose(result['Inv_out'], numpy_inv, rtol=1e-5))

MatInv node implementation: default
Compiling SDFG with Gaussian elimination implementation...
=== Expanding MatInv node with Gaussian elimination ===
Node name: my_matrix_inverse
Created nested SDFG for Gaussian elimination
Failing SDFG saved for inspection in /Users/sophieblock/dev_packages/dace/tutorials/_dacegraphs/failing.sdfgz


KeyError: '__return'

## 8. Testing and Comparing Implementations

Let's compare our two matrix inversion implementations (Gaussian elimination vs. Conjugate Gradient) 
and benchmark them against NumPy:

In [ ]:
import time

# Create a larger test matrix
N_val = 10
A_test = np.random.randn(N_val, N_val).astype(np.float64)
# Make it well-conditioned for numerical stability
A_test = A_test @ A_test.T + N_val * np.eye(N_val)  # symmetric positive-definite

# Function to measure execution time
def time_execution(func, **kwargs):
    start = time.time()
    result = func(**kwargs)
    end = time.time()
    return result, end - start

# 1. Test with Gaussian elimination implementation
sdfg_gaussian = dace.SDFG('invert_matrix_gaussian')
state_gaussian = sdfg_gaussian.add_state()

sdfg_gaussian.add_array('A', shape=[N_val, N_val], dtype=dace.float64)
sdfg_gaussian.add_array('Inv_out', shape=[N_val, N_val], dtype=dace.float64)

A_read_g = state_gaussian.add_read('A')
Inv_write_g = state_gaussian.add_write('Inv_out')

inv_node_g = MatInv("gaussian_inverse")
inv_node_g.implementation = 'gaussian'  # Explicitly choose Gaussian implementation
state_gaussian.add_node(inv_node_g)

state_gaussian.add_edge(A_read_g, None, inv_node_g, 'A', dace.Memlet('A'))
state_gaussian.add_edge(inv_node_g, 'Inv', Inv_write_g, None, dace.Memlet('Inv_out'))

gaussian_func = sdfg_gaussian.compile()
gaussian_result, gaussian_time = time_execution(gaussian_func, A=A_test)

# 2. Test with Conjugate Gradient implementation
sdfg_cg = dace.SDFG('invert_matrix_cg')
state_cg = sdfg_cg.add_state()

sdfg_cg.add_array('A', shape=[N_val, N_val], dtype=dace.float64)
sdfg_cg.add_array('Inv_out', shape=[N_val, N_val], dtype=dace.float64)

A_read_cg = state_cg.add_read('A')
Inv_write_cg = state_cg.add_write('Inv_out')

inv_node_cg = MatInv("cg_inverse")
inv_node_cg.implementation = 'iterative'  # Explicitly choose CG implementation
state_cg.add_node(inv_node_cg)

state_cg.add_edge(A_read_cg, None, inv_node_cg, 'A', dace.Memlet('A'))
state_cg.add_edge(inv_node_cg, 'Inv', Inv_write_cg, None, dace.Memlet('Inv_out'))

cg_func = sdfg_cg.compile()
# Don't need to pass maxiter as it's now a constant in the expanded SDFG
cg_result, cg_time = time_execution(cg_func, A=A_test)

# 3. Test with NumPy's built-in inversion
numpy_result, numpy_time = time_execution(np.linalg.inv, A_test)

# Compare results
print("=== Performance Comparison ===")
print(f"Matrix size: {N_val}x{N_val}")
print(f"Gaussian elimination time: {gaussian_time:.6f} seconds")
print(f"Conjugate Gradient time:   {cg_time:.6f} seconds")
print(f"NumPy inversion time:      {numpy_time:.6f} seconds")

print("\n=== Accuracy Comparison ===")
# Compute errors as Frobenius norm of (computed_inv - true_inv)
gaussian_error = np.linalg.norm(gaussian_result['Inv_out'] - numpy_result)
cg_error = np.linalg.norm(cg_result['Inv_out'] - numpy_result)
print(f"Gaussian elimination error vs NumPy: {gaussian_error:.6e}")
print(f"Conjugate Gradient error vs NumPy:   {cg_error:.6e}")

# Check if A * A^(-1) ≈ I
gaussian_identity = A_test @ gaussian_result['Inv_out']
cg_identity = A_test @ cg_result['Inv_out']
numpy_identity = A_test @ numpy_result

gaussian_identity_error = np.linalg.norm(gaussian_identity - np.eye(N_val))
cg_identity_error = np.linalg.norm(cg_identity - np.eye(N_val))
numpy_identity_error = np.linalg.norm(numpy_identity - np.eye(N_val))

print(f"Gaussian A*A^(-1) error: {gaussian_identity_error:.6e}")
print(f"CG A*A^(-1) error:       {cg_identity_error:.6e}")
print(f"NumPy A*A^(-1) error:    {numpy_identity_error:.6e}")

## Conclusion

In this tutorial, we explored DaCe's Library Node system for operator abstractions in the IR. We:

1. **Examined built-in library nodes** like MatMul and their properties
2. **Created a custom MatInv library node** for matrix inversion
3. **Implemented two different expansions**:
   - Direct method (Gaussian elimination)
   - Iterative method (Conjugate Gradient)
4. **Demonstrated implementation switching** for performance portability
5. **Compared performance** across implementations

The key insight is the separation between:
- **What**: The semantic contract established by the LibraryNode (inputs, outputs, properties)
- **How**: The concrete implementations provided by expansions

This separation enables:
- Algorithm portability across hardware
- Performance optimization through implementation selection
- High-level semantic reasoning about operations

### Further Reading:
- DaCe documentation on library nodes
- Matrix inversion algorithms and their implementations
- High-Performance Linear Algebra libraries (MKL, cuBLAS, etc.)